# Sistema criptográfico RSA (Rivest, Shamir y Adleman).
## Autor: Ricardo Andrés Calvo Méndez

Este sistema criptográfico de clave pública es el más utilizado en el proceso de firmado digital, se caracteríza por ultiliza el problema de factorización de números enteros muy grandes como principal medida de seguridad. Cuando se desea el envío de un mensaje, el emisor debe buscar la clave pública para cifrar con esta el mensaje y posterior a esto el receptor desencripta el mensaje haciendo uso de la clave privada, la cual no debe compartirse por ningún motivo para preservar la seguridad.

El paradigma de la computación cuántica ha estado en proceso de desarrollo durante los ultimos años, pero actualmente ya se cuantan con algoritmos que pueden factorizar en un tiempo razonable los grandes números de la clave publica y privada. Se espera que en los próximos años existan más algoritmos cuánticos que solucionen el problema convirtiendo el sistema RSA en un sistema obsoleto.

***


### ¿Cómo implementar este sistema de encriptación?

***

#### Generación de claves pública y privada.

Para la generación de la clave publica y privada se siguen los siguientes pasos:

1. Se elijen dos números primos $p$ y $q$ diferentes, entre más grandes sean los dos números mas dificil será el proceso de factorización.
2. Se calcula en número $n$ como el producto de los números $p$ y $q$. ($n = pq$)
3. Se calcula el valor de la función $\phi(n)$ de Euler.
4. Se escoje un número $e \in \mathbb{Z}^{+}$ menor que $\phi(n)$ que sea primo relativo con este ($MCD(e, \phi(n)) = 1$).
5. Finalmente se determina un número $d$ que satisfaga $e \cdot d \equiv 1 \mod{\phi(n)}$

La clave pública corresponde a la dupla $(n,e)$ donde $n$ es el modulo y $e$ es el exponente de cifrado, por otra parte la clave privada será la dupla $(n,d)$ donde ahora $d$ es el exponente de descifrado. Ahora se tienen dos procedimientos, la encriptación y la desencriptación los cuales solo se puden hacer con el conocimiento de tanto la clave pública como de la clave privada.

***

#### Encriptación

Para el cifrado se requiere el uso de la clave pública, esta debe ser enviada por el receptor al emisor, la clave privada debe ser guardada en total secreto por el receptor. Suponiendo que el mensaje a enviar es $m$ con $m < n$ y $MCD(m, n) = 1$, el emisor debe calcular un valor $c$ que cumpla lo siguiente:

<center>$c\equiv m^{e}\mod{n}$</center>

Este será el mensaje que el emisor le envía al receptor.

***

#### Desencriptación

Finalmente el receptor puede recuperar el mensaje $m$ usando la clave privada en la siguiente operación.

<center>$m \equiv c^{d}\mod{n}$</center>

De esta manerá el mensaje fue correctamente desencriptado.

***

### ¿Por qué funciona?

***

En el proceso de generación de claves, se utiliza el modulo ($n$) el cual se sabe que es producto de dos números primos ($p$ y $q$), factorizar el número $n$ requiere un costo computacional muy grande y en esto es lo que radica la seguridad de este algoritmo. la recuperación del mensaje a través del uso de las llaves pública y privada es posible gracias a las propiedades de la aritmética modular en el campo de la matemática discreta.

Utilizando las propiedades de la función de Euler, el Teorema de Euler y el Teorema del resto chino, se puede demostrar el siguiente lema:

<center>$x \equiv x^{ed}\mod{n}, \forall x \in \mathbb{Z}_{n}$</center>

Donde $e$ es el exponente de cifrado, $d$ es el exponente de descifrado y $n$ es el módulo de la encriptación ($n = p \cdot q$)

Cuando se encripta el mensaje $m$ y se envía el código $c$, se tiene garantizado que $c\equiv m^{e}\mod{n}$ y al momento de de desencriptar se tiene la siguiente operación:

<center>
    $c \equiv m^e \mod{n}$ <br/>
    $c^{d} \equiv m^{ed} \mod{n}$
</center>

Pero utilizando el lema, se sabe que $m = c^d$ y así queda demostrado que la clave privada sirve perfectamente para recuperar el mensaje después de ser encriptado usando la clave pública.

***

### Ejemplo e implementación del algorítmo

***

Este algoritmo se va a implementar mediante el lenguaje Python. Inicialmente se tendrá un método *main*, varios métodos con operaciones independientes y dos clases: *PublicKey* y *PrivateKey*. Estas dos últimas clases se definen de la siguiente manera:

#### Clase *PublicKey*

In [1]:
class PublicKey:

    def __init__(self, module, exponent):
        self.module = module
        self.exponent = exponent

#### Clase *Privatekey*

In [2]:
class PrivateKey:

    def __init__(self, module, exponent):
        self.module = module
        self.exponent = exponent

Ahora se deben escojer dos números primos $p$ y $q$, estos no deben tener una diferencia muy pequeña ya que la *Factorización de Fermat* sería un método sencillo para factorizar dichos números. En la práctica se utilizan números de alrededor 300 digitos, pero para este ejemplo se elegirán primos aleatorios de 4 digitos, Para ello se utilizará el *Test de primalidad de Fermat*.

#### Test de primalidad de Fermat.

In [3]:
k = 20

In [4]:
from random import randint

def fermat_test(value):
    for i in range(1, k):
        a = randint(2, value - 1)
        if pow(a, value - 1, value) != 1:
            return False
    return True

Es esta función $k$ indicará la cantidad de veces que se realizará el test sobre el número dado *value*, si un este pasa las $k$ veces el test, se tomará como primo. Cuanto más grande sea $k$, se podrá confiar más que el número dado es primo.
La complejidad del algoritmo es $O(k)$, por lo que una cantidad muy grande de comprobaciones supondrá un alto costo computacional.

Este test funciona gracias al *Pequeño Teorema de Fermat* el cual se enuncia de la siguiente manera:

#### Pequeño Teorema de Fermat.

<center>$ a^{p-1} \equiv 1 \mod{p} $</center>

Para todo $a \in \mathbb{N}$ y para todo $p$ número prímo.

Todos los números primos cumplen esa propiedad pero eso **NO** implica que algunos números compuestos no puedan cumplir la congruencia modular para algunos valores especificos de $a$, es por esto que al hacer $k$ pruebas, se pueden descartar dichos números compuestos.

Ahora se generarán primos aleatorios usando la siguiente función:

In [5]:
def random_prime(magnitude_order):
    flag = False
    while not flag:
        n = randint(pow(10, magnitude_order - 1), pow(10, magnitude_order))
        flag = fermat_test(n)
    return n

El argumento *magnitude_order* indicará el orden de magnitud de los primos a generar. A continuación se ejecuta la función sin olvidar que $q$ y $q$ deben ser diferentes.

In [6]:
magnitude_order = 4

In [7]:
p = random_prime(magnitude_order)
print("p =", p)

p = 3733


In [8]:
while True:
    q = random_prime(magnitude_order)
    if p != q:
        break
print("q =", q)

q = 5479


Bien, ya con los números primos generados se va a calcular el valor de $n$ o del módulo.

In [9]:
n = p * q
print("n =", n)

n = 20453107


A continuación se calculará el valor de $\phi (n)$ (El cual será utilizado para calcular los exponentes de cifrado y descifrado), para ello se utilizarán dos propiedades de dicha función:

**1:** Si $p$ es un número primo, entonces $\phi(p) = p-1$

**2:** Si $p$ y $q$ son primos relativos, entonces $\phi(pq) = \phi(p)\phi(q)$

Cómo $n$ es producto de dos números primos, se sabe que $\phi(n) = (p-1)(q-1)$

In [10]:
phi_n = (p-1)*(q-1)
print("Phi(n) = ", phi_n)

Phi(n) =  20443896


En este punto se deben calcular los exponentes de cifrado y descifrado. Para el exponente de cifrado ($e$) se debe elegir un número aleatorio que sea primo relativo con $\phi(n)$, en pocas palabras, que $MCD(e, \phi(n)) = 1$. Para el calculo de este máximo común divisor se utilizará el *Algoritmo de Euclides*.

#### Algoritmo de Euclides.

In [11]:
def euclidean_algorithm(a, b):
    if a < b:
        temp = a
        a = b
        b = temp
    while True:
        int(a / b)
        remainder = a % b
        a = b
        b = remainder
        if remainder <= 0:
            return a

Este algoritmo descrito por primera vez por Euclides es muy eficiente $O(\log{ab})$ y se aprovecha de la siguiente propiedad:

Sea $a,b \in \mathbb{N}$ con $a \geq b$ y $a = b \cdot c + r$ entonces:

<center> $MCD(a , b) = MCD(b, r)$ </center>

En pocas palabras, el máximo común divisor entre dos números es igual a el máximo común divisor entre el mas pequeño de ellos y el residuo de su cociente.

Es así como se comprueba eficientemente que un número aleatorio $e$ sea primo relativo con $\phi(n)$. Para generar el exponente de cifrado se utilizará la siguiente función:

In [12]:
def encrypt_exponent(phi_n):
    while True:
        e = randint(2, phi_n - 1)
        if euclidean_algorithm(e, phi_n) == 1:
            return e

Generando $e$ se tiene lo siguente:

In [13]:
e = encrypt_exponent(phi_n)
print("e = ", e)

e =  3228053


Ahora solo queda generar el exponente de descifrado ($d$) y para ello se requiere del uso del *Algoritmo extendido de Euclides*. El principio de este es el mismo que el anterior, pero la extensión se basa en el calculo del máximo común divisor como una *combinación lineal*. En el algoritmo se debe guardar cada paso a paso de la busqueda del máximo común divisor para luego usar esa información y encontrar la combinación lineal deseada.

#### Algoritmo externido de Euclides.

In [14]:
def extended_euclidean_algorithm(number, module):
    processes = []
    d = 0
    while True:
        processes.append([])
        processes[len(processes) - 1].append(module)
        processes[len(processes) - 1].append(number)
        quotient = int(module / number)
        processes[len(processes) - 1].append(quotient)
        remainder = module % number
        processes[len(processes) - 1].append(remainder)
        module = number
        number = remainder
        if remainder <= 0:
            break
    basis_a = processes[len(processes)-2][0]
    factor_a = 1
    factor_b = -processes[len(processes)-2][2]
    i = 3
    while len(processes) - i >= -1:
        basis_b = basis_a
        factor_a = factor_b
        basis_a = processes[len(processes) - i][0]
        factor_b = int((1 - basis_a * factor_a)/basis_b)
        i = i + 1
    return factor_a

Este algorítmo funciona para el calculo de $d$ ya que se desea encontrar el inverso multiplicativo modular de $e$ respecto a el módulo $\phi(n)$. Esto se hace teniendo en cuenta la siguiente propiedad:

Sea $d$ el inverso multiplicativo modular de $e$, es decir:

<center>$e \cdot d \equiv 1 \mod{\phi(n)}$</center>

Y sabiendo por propiedades del algoritmo de Euclides que:

<center>$ 1 = x\cdot e + y\cdot \phi(n) $</center>

Se puede afirmar que:

<center>$d = x$</center>

Vale recordar que $e$ tiene inverso multiplicativo modular ya que $\phi(n)$ y $e$ son primos relativos. (Esto es una equivalencia lógica, es decir, un número tiene inverso multiplicativo modular si y solo si ese número y el módulo son primos relativos)

Para el calculo de $d$ se utlizará la siguiente función teniendo en cuenta que $d$ no puede ser menor que cero aunque el algoritmo extendido de Euclides arroje un valor negativo:

In [15]:
def decrypt_exponent(e, phi_n):
    d = extended_euclidean_algorithm(e, phi_n)
    if d > 0:
        return d
    else:
        return d + phi_n

Calculando $d$ se tiene:

In [16]:
d = decrypt_exponent(e, phi_n)
print("d = ", d)

d =  16755005


Ahora solo queda definir las claves pública y privada. En el caso de la clave pública, esta será el par $(n, e)$

In [17]:
public_key = PublicKey(n, e)
print("Clave pública: (" + str(public_key.module) + ", " + str(public_key.exponent) + ")")

Clave pública: (20453107, 3228053)


Y la clave privada será el par $(n, d)$

In [18]:
private_key = PrivateKey(n, d)
print("Clave privada: (" + str(private_key.module) + ", " + str(private_key.exponent) + ")")

Clave privada: (20453107, 16755005)


****

Ahora se va a elegir una cadena de caracteres como mensaje a enviar, como este algoritmo solo puede enviar números, se va a convertir cada caracter a su equivalente en la secuencia ASCII; se va a encriptar cada caracter de la cadena por separado usando la *clave pública* y así envíar un codigo encriptado por cada caracter.

In [19]:
m = "Discrete Math"

In [20]:
c_list = []
for a in m:
    c_list.append(pow(ord(a), public_key.exponent, public_key.module))
i = 1;
for c in c_list:
    print("c[" + str(i) + "] = " + str(c))
    i = i + 1

c[1] = 8841774
c[2] = 11078965
c[3] = 7630765
c[4] = 11741881
c[5] = 18277371
c[6] = 2314158
c[7] = 14595009
c[8] = 2314158
c[9] = 2823535
c[10] = 7807431
c[11] = 18765285
c[12] = 14595009
c[13] = 8700012


Aqui se utilizó la operación de encriptación, es decir, para cada caracter $a_i \in m$ se encuentra un $c_i$ que satisface:

<center>$ c_i \equiv ASCCI(a_i)^{e} \mod{n} $</center>

Recordando que el par (n, e) corresponde a la clase pública la cual fue enviada previamente por el receptor al emisor.

Ahora, el emisor le envía al receptor la lista con todos los $c$ que corresponden a los caracteres encriptados. Ya con la lista en manos del receptor, este utiliza la *clave privada* para recuperar el mensaje original; hay que tener en cuenta que el receptor debe saber que el mensaje esta codificado según la secuencia ASCCI.

In [21]:
recovered_message = ""
for c in c_list:
    recovered_message += chr(pow(c, private_key.exponent, private_key.module))
print("Mensaje recuperado:", recovered_message)

Mensaje recuperado: Discrete Math


Para recuperar el mensaje se encontro para cada código $c_i$ un $m_i$ que safisface:

<center>$ m_i \equiv (c_i)^{d} \mod{n} $</center>

Donde el par (n, d) correspondw a la clave privada. Ahora con los códigos ASCCI decada caracter, se recuperó la cadena aplicando la funcion inversa $ASCCI^{-1}(m_i) \forall_{i}$ y concatenar.

***